In [1]:
import numpy as np
import pandas as pd
import os

In [6]:
from dotenv import load_dotenv

load_dotenv()

path_to_dataset = os.getenv('path_to_provided_data')
path_to_extracted = os.getenv('path_to_extracted_data')

In [3]:
# load the performance data

data = pd.read_csv(path_to_dataset+'performance_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   game_id       905 non-null    object 
 1   preview_link  905 non-null    object 
 2   ER            905 non-null    float64
 3   CTR           905 non-null    float64
dtypes: float64(2), object(2)
memory usage: 28.4+ KB


In [7]:
# write a txt file for path to creative assets

df = pd.DataFrame(columns=['start_frame', 'end_frame'])

df['start_frame'] = data['game_id'].apply(lambda x: path_to_extracted + x + '/start_frame.png' )
df['end_frame'] = data['game_id'].apply(lambda x: path_to_extracted + x + '/end_frame.png' )

np.savetxt('../data/start_frames.txt', df['start_frame'].values, delimiter=',', fmt='%s')
np.savetxt('../data/end_frames.txt', df['end_frame'].values, delimiter=',', fmt='%s')